<a href="https://colab.research.google.com/github/C8XY66/GIN/blob/main/GIN_CV_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
!pip install pytorch-lightning

In [ ]:
!pip install git+https://github.com/optuna/optuna.git
!pip install optuna-dashboard

In [ ]:
import os.path as osp
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
from typing import Optional
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold


import logging
#logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
#logging.getLogger("lightning").setLevel(logging.ERROR)
#logging.getLogger("lightning").setLevel(logging.CRITICAL)
#logging.getLogger('lightning').setLevel(0)
#logging.getLogger("lightning.pytorch").setLevel(logging.ERROR)
logging.getLogger("pytorch_lightning.utilities.rank_zero").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.accelerators.cuda").setLevel(logging.WARNING)
#logging.getLogger("pytorch_lightning.callbacks.early_stopping").setLevel(logging.WARNING)



import torch
import torch.nn.functional as F
from torchmetrics import Accuracy

import torch_geometric.transforms as T
from torch_geometric.data.lightning import LightningDataset
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GIN, MLP, global_add_pool
from torch_geometric.data import InMemoryDataset

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

import optuna
from optuna.integration import PyTorchLightningPruningCallback
from optuna.visualization.matplotlib import plot_contour, plot_edf, plot_intermediate_values, plot_optimization_history, plot_parallel_coordinate, plot_param_importances, plot_slice


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
DIR = "/content/gdrive/My Drive/ColabNotebooks/" 

# Check for CUDA system support and use GPU if available otherwise run on CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # device = Context-manager that changes the selected device
print(device)

cuda


In [ ]:
# Hyperparameters

DATASET_NAME = 'NCI1'
num_layers = 5
lr = 0.01 
EPOCHS = 200 #final = 1000
SEED = 42
N_SPLITS = 5
REP = 3

In [ ]:
class Model(pl.LightningModule):
    def __init__(self, in_channels: int, out_channels: int,
                 hidden_channels: int, dropout, num_layers=num_layers):
        super().__init__()
        self.gnn = GIN(in_channels, hidden_channels, num_layers,
                       dropout=dropout, jk='cat')

        self.classifier = MLP([hidden_channels, hidden_channels, out_channels],
                              norm="batch_norm", dropout=dropout)

        self.train_acc = Accuracy(task='multiclass', num_classes=out_channels)
        self.val_acc = Accuracy(task='multiclass', num_classes=out_channels)
        self.test_acc = Accuracy(task='multiclass', num_classes=out_channels)

    def forward(self, x, edge_index, batch):
        x = self.gnn(x, edge_index)
        x = global_add_pool(x, batch)
        x = self.classifier(x)
        return x

    def training_step(self, data, batch_idx):
        y_hat = self(data.x, data.edge_index, data.batch)
        loss = F.cross_entropy(y_hat, data.y)
        self.train_acc(y_hat.softmax(dim=-1), data.y)
        self.log('train_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train_acc', self.train_acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, data, batch_idx):
        y_hat = self(data.x, data.edge_index, data.batch)
        loss = F.cross_entropy(y_hat, data.y)
        self.val_acc(y_hat.softmax(dim=-1), data.y)
        self.log('val_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val_acc', self.val_acc, prog_bar=True, on_step=False, on_epoch=True)

    def test_step(self, data, batch_idx):
        y_hat = self(data.x, data.edge_index, data.batch)
        loss = F.cross_entropy(y_hat, data.y)
        self.test_acc(y_hat.softmax(dim=-1), data.y)
        self.log('test_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('test_acc', self.test_acc, prog_bar=True, on_step=False,on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.01)


In [ ]:
class GraphDataModule(pl.LightningDataModule):
    def __init__(self, dataset_name, n_splits=10, fold=0):
        super().__init__()
        self.dataset_name = dataset_name
        self.n_splits = n_splits
        self.fold = fold

    def prepare_data(self):    
        self.dataset = TUDataset(root='data/TUDataset', name=self.dataset_name)
        #self.dataset = self.dataset[:500] #for quick experiments
        self.skf = StratifiedKFold(n_splits=self.n_splits)

    def setup(self, stage: Optional[str] = None, fold: int = 0, batch_size: int = 32):
        self.fold = fold
        self.batch_size = batch_size
        y = [data.y.item() for data in self.dataset]

        train_indices, test_indices = list(self.skf.split(torch.zeros(len(y)), y))[self.fold]
        train_dataset = self.dataset[train_indices]
        
        num_val = int(len(train_dataset) * 0.1)
        num_train = len(train_dataset) - num_val
        
        self.train_dataset, self.val_dataset = torch.utils.data.random_split(train_dataset, [num_train, num_val])
        self.test_dataset = self.dataset[test_indices]
      
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)

    @property
    def num_node_features(self):
        return self.dataset.num_node_features

    @property
    def num_classes(self):
        return self.dataset.num_classes


In [ ]:
def objective(trial, datamodule, epochs): 

    # Optimise hyperparameters
    hidden_channels = trial.suggest_categorical('hidden_channels', [16, 32])
    batch_size = trial.suggest_categorical('batch_size', [32, 128])
    dropout = trial.suggest_categorical('dropout', [0.0, 0.5])

    # Model and DataModule
    datamodule.setup(fold=0, batch_size=batch_size)
    model = Model(in_channels=datamodule.num_node_features, out_channels=datamodule.num_classes, hidden_channels=hidden_channels, dropout=dropout)

    # Training
    pruning_callback = PyTorchLightningPruningCallback(trial, monitor="val_acc") #from optuna-pl-integration
    early_stopping = EarlyStopping(monitor="val_acc", mode="max", patience=10, verbose=True) #final patience = 500 for acc and loss
    
    trainer = pl.Trainer(
        default_root_dir=DIR,
        callbacks=[pruning_callback, early_stopping], 
        max_epochs=epochs,
        log_every_n_steps=5, 
        logger=TensorBoardLogger(save_dir=DIR),
        enable_progress_bar=False,
        enable_model_summary=False,
    )

    hyperparameters = dict(hidden_channels=hidden_channels, batch_size=batch_size, epochs=epochs, dropout=dropout)
    trainer.logger.log_hyperparams(hyperparameters)    
    
    trainer.fit(model, datamodule=datamodule)

    return trainer.callback_metrics['val_acc'].item()
    

In [ ]:
def retrain_and_evaluate(model, datamodule, epochs):
    early_stopping = EarlyStopping(monitor="val_acc", mode="max", patience=10, verbose=True)
    trainer = pl.Trainer(
        default_root_dir=DIR,
        callbacks=[early_stopping],
        max_epochs=epochs,
        log_every_n_steps=5,
        logger=TensorBoardLogger(save_dir=DIR),
        enable_progress_bar=False,
        enable_model_summary=False,
    )
    trainer.fit(model, datamodule=datamodule)
    trainer.test(model, datamodule=datamodule)

    return trainer.callback_metrics['test_acc'].item()

In [ ]:
if __name__ == '__main__': 

    datamodule = GraphDataModule(dataset_name=DATASET_NAME)
  
    overall_performances = []
    for r in range(REP):
        datamodule.prepare_data()
        fold_performances = []
        for fold in range(N_SPLITS):
            # Create a new study object for each fold
            study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner(), sampler=optuna.samplers.TPESampler(seed=SEED),)
            datamodule.setup("fit", fold)
            study.optimize(lambda trial: objective(trial, datamodule, EPOCHS), n_trials=8)
            print(f"Best trial for fold {fold}: {study.best_trial.value}")

            # Retrain the model with the best hyperparameters
            best_params = study.best_trial.params
            model = Model(in_channels=datamodule.num_node_features,
                          out_channels=datamodule.num_classes,
                          hidden_channels=best_params['hidden_channels'],
                          dropout=best_params['dropout'])
            datamodule.setup("fit", fold, batch_size=best_params['batch_size'])
            test_acc = retrain_and_evaluate(model, datamodule, EPOCHS)
            fold_performances.append(test_acc)

        avg_performance = np.mean(fold_performances)
        print(f"Average performance for repetition {r}: {avg_performance}")
        overall_performances.append(avg_performance)

    print(f"Overall average performance: {np.mean(overall_performances)}")


[I 2023-04-18 16:56:14,836] A new study created in memory with name: no-name-c5b311ef-2d0e-4624-9dbd-fe7a315431fa
[I 2023-04-18 16:56:51,186] Trial 0 finished with value: 0.7588075995445251 and parameters: {'hidden_channels': 32, 'batch_size': 32, 'dropout': 0.0}. Best is trial 0 with value: 0.7588075995445251.
[I 2023-04-18 16:57:21,149] Trial 1 finished with value: 0.7506775259971619 and parameters: {'hidden_channels': 32, 'batch_size': 128, 'dropout': 0.5}. Best is trial 0 with value: 0.7588075995445251.
[I 2023-04-18 16:57:59,081] Trial 2 finished with value: 0.7317073345184326 and parameters: {'hidden_channels': 16, 'batch_size': 128, 'dropout': 0.5}. Best is trial 0 with value: 0.7588075995445251.
[I 2023-04-18 16:58:39,246] Trial 3 finished with value: 0.696476936340332 and parameters: {'hidden_channels': 16, 'batch_size': 32, 'dropout': 0.5}. Best is trial 0 with value: 0.7588075995445251.
[I 2023-04-18 16:59:25,466] Trial 4 finished with value: 0.718157172203064 and parameters

Best trial for fold 0: 0.7588075995445251


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.6326034069061279     │
│         test_loss         │    0.6700522899627686     │
└───────────────────────────┴───────────────────────────┘

[I 2023-04-18 17:00:08,246] A new study created in memory with name: no-name-9acd3593-8f95-4d60-beea-8a375fb1a175
[I 2023-04-18 17:00:24,361] Trial 0 finished with value: 0.631436288356781 and parameters: {'hidden_channels': 32, 'batch_size': 32, 'dropout': 0.0}. Best is trial 0 with value: 0.631436288356781.
[I 2023-04-18 17:00:51,860] Trial 1 finished with value: 0.7317073345184326 and parameters: {'hidden_channels': 32, 'batch_size': 128, 'dropout': 0.5}. Best is trial 1 with value: 0.7317073345184326.
[I 2023-04-18 17:01:35,127] Trial 2 finished with value: 0.783197820186615 and parameters: {'hidden_channels': 16, 'batch_size': 128, 'dropout': 0.5}. Best is trial 2 with value: 0.783197820186615.
[I 2023-04-18 17:02:02,599] Trial 3 finished with value: 0.696476936340332 and parameters: {'hidden_channels': 16, 'batch_size': 32, 'dropout': 0.5}. Best is trial 2 with value: 0.783197820186615.
[I 2023-04-18 17:02:32,325] Trial 4 finished with value: 0.6991869807243347 and parameters: {'

Best trial for fold 1: 0.783197820186615


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.65450119972229      │
│         test_loss         │    0.6553006768226624     │
└───────────────────────────┴───────────────────────────┘

[I 2023-04-18 17:54:36,546] A new study created in memory with name: no-name-f1766f17-a121-4661-8fe2-87147b2caf79
[I 2023-04-18 17:55:06,344] Trial 0 finished with value: 0.6585366129875183 and parameters: {'hidden_channels': 32, 'batch_size': 32, 'dropout': 0.0}. Best is trial 0 with value: 0.6585366129875183.
[I 2023-04-18 17:55:43,032] Trial 1 finished with value: 0.7696477174758911 and parameters: {'hidden_channels': 32, 'batch_size': 128, 'dropout': 0.5}. Best is trial 1 with value: 0.7696477174758911.
[I 2023-04-18 17:56:24,227] Trial 2 finished with value: 0.7100270986557007 and parameters: {'hidden_channels': 16, 'batch_size': 128, 'dropout': 0.5}. Best is trial 1 with value: 0.7696477174758911.
[I 2023-04-18 17:57:01,647] Trial 3 finished with value: 0.7506775259971619 and parameters: {'hidden_channels': 16, 'batch_size': 32, 'dropout': 0.5}. Best is trial 1 with value: 0.7696477174758911.
[I 2023-04-18 17:57:22,743] Trial 4 finished with value: 0.7154471278190613 and paramete

Best trial for fold 4: 0.7994579672813416


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5936739444732666     │
│         test_loss         │    0.6353301405906677     │
└───────────────────────────┴───────────────────────────┘

Average performance for repetition 2: 0.6428223848342896
Overall average performance: 0.6306569298108419
